In [ ]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Create the Bayesian Network
model = BayesianNetwork([
    ('IQ', 'Marks'),      # IQ affects Marks
    ('ExamLevel', 'Marks'),  # ExamLevel affects Marks
    ('Marks', 'Admission'),  # Marks affects Admission
    ('IQ', 'Aptitude')    # IQ affects Aptitude
])

# Step 2: Define the Conditional Probability Tables (CPTs)

# P(IQ)
cpd_iq = TabularCPD(variable='IQ', variable_card=2,
                    values=[[0.7],  # P(IQ=High)
                            [0.3]])  # P(IQ=Low)

# P(ExamLevel)
cpd_exam_level = TabularCPD(variable='ExamLevel', variable_card=2,
                            values=[[0.6],  # P(ExamLevel=Easy)
                                    [0.4]])  # P(ExamLevel=Difficult)

# P(Marks | IQ, ExamLevel)
cpd_marks = TabularCPD(variable='Marks', variable_card=2,
                       values=[[0.9, 0.7, 0.6, 0.1],  # P(Marks=Good)
                               [0.1, 0.3, 0.4, 0.9]],  # P(Marks=Bad)
                       evidence=['IQ', 'ExamLevel'],
                       evidence_card=[2, 2])

# P(Aptitude | IQ)
cpd_aptitude = TabularCPD(variable='Aptitude', variable_card=2,
                          values=[[0.8, 0.4],  # P(Aptitude=High)
                                  [0.2, 0.6]],  # P(Aptitude=Low)
                          evidence=['IQ'], evidence_card=[2])

# P(Admission | Marks)
cpd_admission = TabularCPD(variable='Admission', variable_card=2,
                           values=[[0.9, 0.4],  # P(Admission=Yes)
                                   [0.1, 0.6]],  # P(Admission=No)
                           evidence=['Marks'], evidence_card=[2])

# Step 3: Add CPTs to the model
model.add_cpds(cpd_iq, cpd_exam_level, cpd_marks, cpd_aptitude, cpd_admission)

# Step 4: Check the model validity
if model.check_model():
    print("The Bayesian Network is valid.")

# Step 5: Perform Inference
inference = VariableElimination(model)

# Query 1: What is the probability of Admission given Marks are Good?
result = inference.query(variables=['Admission'], evidence={'Marks': 0})
print("\nProbability of Admission given Marks are Good:")
print(result)

# Query 2: What is the probability of Marks given IQ is High and ExamLevel is Easy?
result = inference.query(variables=['Marks'], evidence={'IQ': 0, 'ExamLevel': 0})
print("\nProbability of Marks given IQ is High and ExamLevel is Easy:")
print(result)

# Query 3: What is the probability of Aptitude given IQ is Low?
result = inference.query(variables=['Aptitude'], evidence={'IQ': 1})
print("\nProbability of Aptitude given IQ is Low:")
print(result)


The Bayesian Network is valid.

Probability of Admission given Marks are Good:
+--------------+------------------+
| Admission    |   phi(Admission) |
+==============+==================+
| Admission(0) |           0.9000 |
+--------------+------------------+
| Admission(1) |           0.1000 |
+--------------+------------------+

Probability of Marks given IQ is High and ExamLevel is Easy:
+----------+--------------+
| Marks    |   phi(Marks) |
+==========+==============+
| Marks(0) |       0.9000 |
+----------+--------------+
| Marks(1) |       0.1000 |
+----------+--------------+

Probability of Aptitude given IQ is Low:
+-------------+-----------------+
| Aptitude    |   phi(Aptitude) |
+=============+=================+
| Aptitude(0) |          0.4000 |
+-------------+-----------------+
| Aptitude(1) |          0.6000 |
+-------------+-----------------+

Probability of Admission given IQ is High and ExamLevel is tough and marks are bad:
+--------------+------------------+
| Admissio